<a href="https://colab.research.google.com/github/pierrelouisbescond/medium_articles/blob/main/medium_sharpness_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### *PEP8 checked*

In [ ]:
µimport numpy as np
import pandas as pd
import glob
from PIL import Image
import plotly.express as px


# How could we evaluate the sharpness of a picture?

In [ ]:
folder_path = "./drive/MyDrive/Medium/sharpness_evaluation/"
selected_pictures = ["image065.jpg", "image000.jpg", "image064.jpg"]
idx = 0

for picture in selected_pictures:
    im = Image.open(folder_path+picture).convert('L')
    array = np.asarray(im, dtype=np.int32)
    print("Image_{0}: Min: {1:3} | Mean : {2:3.0f} | Max : {3:3.0f}".format(idx, array.min(), array.mean(), array.max()))
    idx += 1


Image_0: Min:  69 | Mean : 146 | Max : 187
Image_1: Min: 107 | Mean : 155 | Max : 192
Image_2: Min: 107 | Mean : 151 | Max : 181


# The gradient to the rescue!

In [ ]:
zeros_array = np.zeros((3, 3))
random_array = [[1, 3, 0], [2, 1, 0], [0, 1, 0]] #np.random.randint(5,size=(3,3))
normal_array = [[0, 1, 0], [1, 5, 1], [0, 1, 0]]


In [ ]:
print ("zeros_array: {:.2f}".format(np.average(abs(np.gradient(zeros_array)[0]) + abs(np.gradient(zeros_array)[1]))))
print ("random_array: {:.2f}".format(np.average(abs(np.gradient(random_array)[0]) + abs(np.gradient(random_array)[1]))))
print ("normal_array: {:.2f}".format(np.average(abs(np.gradient(normal_array)[0]) + abs(np.gradient(normal_array)[1]))))


zeros_array: 0.00
random_array: 1.89
normal_array: 2.67


# Let's go big!

In [ ]:
files_in_folder = glob.glob(folder_path+"*")
nb_of_files_in_folder = len(files_in_folder)

sharpness_results = pd.DataFrame(np.zeros((nb_of_files_in_folder, 2)), columns=["filename", "sharpness_value"])

for idx, file in enumerate(files_in_folder):
    img = Image.open(file).convert('L')
    array = np.asarray(img, dtype=np.int32)
    yaxis_gradient, xaxis_gradient = np.gradient(array)
    sharpness = np.average(abs(yaxis_gradient)+abs(xaxis_gradient))
    sharpness_results.iloc[idx, 0] = file[44:-4]
    sharpness_results.iloc[idx, 1] = sharpness

sharpness_results = sharpness_results.set_index("filename").sort_index()


In [ ]:
fig = px.scatter(y=sharpness_results["sharpness_value"])
fig.show()
